In [1]:
import os, json, time
import gc
from IPython.display import display, Markdown

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM, TrainingArguments
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import FAISS, Chroma
# Text embedding / Texxt Splitter for RAG 
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from datasets import Dataset, DatasetDict, load_dataset

# Adv RAG library 

# for evaluate  LLM 
import evaluate # require online 
# from deepeval.metrics import GEval
# from deepeval.test_case import LLMTestCase
# from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric , ContextualRelevancyMetric, ContextualRecallMetric
import pytest
# import trulens

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
